# ZHCPA Dingtalk

Resources:
- [Dingtalk Dev Docs](https://open.dingtalk.com/document/orgapp/)
- [Dingtalk Backend API Explorer](https://open-dev.dingtalk.com/apiExplorer#/?devType=org)
- [Dingtalk Frontend API Explorer](https://open-dev.dingtalk.com/apiExplorer#/jsapi?api=device.notification.extendModal)
- [Dingtalk Admin Dashboard](https://oa.dingtalk.com/admin/portal/oa#)

Schema:
![](assets/schema.png)

In [ ]:
from op.supabase import supabase
import pandas as pd

raw_data = supabase.table("daily_attendance_view").select("*").execute().data
raw_data
raw_df = pd.DataFrame(raw_data)
raw_df

In [ ]:
from pprint import pprint
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime, time
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator
import calendar
from matplotlib.lines import Line2D


year = 2023
month = 7
id = None
name = "Tong.Wong"

if not id and not name:
    raise Exception("Either id or name must be provided")

fig, ax = plt.subplots(figsize=(20, 20))

# Set the limits for the x-axis
ax.set_xlim(date(year, month, 1), date(year, month + 1, 1) - timedelta(days=1))

# Set the major ticks for the x-axis
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%A %d-%b-%Y"))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%a %d"))

# Set the limits for the y-axis
start_time = time(0, 0)
end_time = time(23, 59)

ax.set_ylim(
    start_time.hour * 60 + start_time.minute, end_time.hour * 60 + end_time.minute
)

# Set the major ticks for the y-axis (every hour)
ax.yaxis.set_major_locator(MultipleLocator(base=60))
ax.yaxis.set_major_formatter(lambda x, pos: f"{int(x / 60):02d}:{int(x % 60):02d}")

# Set the minor ticks for the y-axis (every 30 minutes)
ax.yaxis.set_minor_locator(MultipleLocator(base=30))

# Get the number of days in the month
num_days = calendar.monthrange(year, month)[1]

# Get opuser's name
if not name:
    name = (
        supabase.table("opuser")
        .select("name")
        .eq("id", id)
        .single()
        .execute()
        .data["name"]
    )

# Get opuser's id
if not id:
    id = (
        supabase.table("opuser")
        .select("id")
        .eq("name", name)
        .single()
        .execute()
        .data["id"]
    )

# Insert bars

attendance_carry_over: int = 0
for i in range(num_days):
    day = i + 1

    row = (
        supabase.table("opuser_daily_attendance_view")
        .select("*")
        .eq("opuser_id", id)
        .eq("attendance_date", date(year, month, day).isoformat())
        .single()
        .execute()
    ).data

    # Insert attendance data (carry over, if any)

    if attendance_carry_over > 0:
        ax.bar(
            x=date(year, month, day),
            height=(attendance_carry_over),
            bottom=0,
            alpha=0.5,
            width=0.6,
            color="#3063a1",
        )

    attendance_carry_over = 0

    # Insert shift data (don't insert if it's a holiday or an off-day)

    shift_start_time = time.fromisoformat(row["shift_start_time"])
    shift_end_time = time.fromisoformat(row["shift_end_time"])

    if not row["is_holiday"] and not row["is_shift_off_day"]:
        ax.bar(
            x=date(year, month, day),
            height=(
                shift_end_time.hour * 60
                + shift_end_time.minute
                - shift_start_time.hour * 60
                + shift_start_time.minute
            ),
            bottom=shift_start_time.hour * 60 + shift_start_time.minute,
            alpha=0.2,
            width=0.8,
            color="#5f5f5f",
        )

    # Insert opuser attendance data

    if not row["is_record_valid"]:
        continue

    check_in_times = row["check_in_times"]
    check_out_times = row["check_out_times"]
    if len(check_in_times) > 1 or len(check_out_times) > 1:
        continue

    # TODO: draw a bar for each atttendance pair

    start_time = datetime.fromisoformat(check_in_times[0])
    end_time = datetime.fromisoformat(check_out_times[0])

    if end_time.day == start_time.day + 1:
        attendance_carry_over = end_time.hour * 60 + end_time.minute
        end_time = datetime(year, month, day, 23, 59)
    elif end_time.day == start_time.day:
        pass
    else:
        raise Exception("No support for > 24 hours attendance yet")

    ax.bar(
        x=date(year, month, day),
        height=(
            end_time.hour * 60
            + end_time.minute
            - start_time.hour * 60
            + start_time.minute
        ),
        bottom=start_time.hour * 60 + start_time.minute,
        alpha=0.5,
        width=0.6,
        color="#3063a1",
    )

# Insert leaves

leaves = (
    supabase.table("monthly_leave_view")
    .select("*")
    .eq("opuser_id", id)
    .eq("month", date(year, month, 1).isoformat())
    .single()
    .execute()
    .data["leaves"]
)

if leaves:
    for leave in leaves:
        leave_record = (
            supabase.table("leave").select("*").eq("id", leave).single().execute().data
        )

        # TODO: handle leave carry-over
        leave_start_time = datetime.fromisoformat(leave_record["leave_start_time"])
        leave_end_time = datetime.fromisoformat(leave_record["leave_end_time"])

        ax.bar(
            x=date(leave_start_time.year, leave_start_time.month, leave_start_time.day),
            height=(
                leave_end_time.hour * 60
                + leave_end_time.minute
                - leave_start_time.hour * 60
                + leave_start_time.minute
            ),
            bottom=leave_start_time.hour * 60 + leave_start_time.minute,
            alpha=0.15,
            width=0.6,
            color="#963939",
        )

ax.invert_yaxis()

plt.gcf().autofmt_xdate()

plt.legend(loc="upper right")

ax.legend(
    [
        Line2D([0], [0], color="#cc5e5e", lw=8, alpha=0.8),
        Line2D([0], [0], color="#2270ca", lw=8, alpha=0.8),
        Line2D([0], [0], color="#a7a7a7", lw=8, alpha=0.8),
    ],
    ["Leave", "Attendance", "Working hours"],
)

plt.title(f"{name}'s {year}-{month} attendance report")

plt.show()